In [7]:
import json
from azure.eventhub import EventHubProducerClient, EventData
from dotenv import load_dotenv
import os
import socket
import random
from faker import Faker
from random import randrange

load_dotenv()
EVENT_HUB_CONN_STRING = os.getenv("EVENT_HUB_CONN_STRING") 
EVENT_HUB_NAME_IMPRESSIONS = os.getenv("EVENT_HUB_NAME_IMPRESSIONS")
EVENT_HUB_NAME_CLICKS = os.getenv("EVENT_HUB_NAME_CLICKS")

eventHubConnString = EVENT_HUB_CONN_STRING
eventHubNameImpressions = EVENT_HUB_NAME_IMPRESSIONS
eventHubNameClicks = EVENT_HUB_NAME_CLICKS

producer_impressions = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameImpressions)
producer_clicks = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameClicks)

hostname = socket.gethostname()

In [22]:

productIds = [680,706,707,708,709,710,711,712]
            #   ,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999]

def generateClickEvent(impressionEvent):
    faker = Faker()
    click = {}
    click["impressionId"] = impressionEvent["impressionId"]
    click["date"] = impressionEvent["date"]
    click["productId"] = impressionEvent["productId"]
    click["browser"] = impressionEvent["browser"]
    click["version"] = impressionEvent["version"]
    click["device"] = impressionEvent["device"]
    click["source"] = impressionEvent["source"]
    click["ip_address"] = impressionEvent["ip_address"]
    return click


def generateImpressionEvent(isUnsupportedBrowser):
    faker = Faker()
    impression = {}
    impression["impressionId"] = faker.uuid4()
    impression["date"] = faker.date_time_between(start_date="-1y", end_date="now").isoformat()
    impression["productId"] = faker.random_element(productIds)
    impression["browser"] = faker.random_element(["Edge", "Chrome", "Safari", "Firefox"])
    impression["version"] = faker.random_element(["10.2", "13.6", "8.6", "8.5", "11.2", "14.6", "6.6", "4.5"])
    impression["device"] = faker.random_element(["mobile", "computer", "tablet", "mobile", "computer"])
    impression["source"] = faker.random_element( ["organic", "bing", "google", "facebook"])
    impression["ip_address"] = faker.ipv4()
    impression["landing_page"] = faker.uri()
    return impression


def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)

def generateEvents(isUnsupportedBrowser = False):
    try:
        while True:
            impressionEvent = generateImpressionEvent(isUnsupportedBrowser)    
            sendToEventsHub(impressionEvent, producer_impressions)
            if random.randint(1, 100) > 80:
                clickEvent = generateClickEvent(impressionEvent)    
                sendToEventsHub(clickEvent, producer_clicks)
    except KeyboardInterrupt:
        producer_impressions.close()
        producer_clicks.close()


In [23]:
generateEvents()

{"impressionId": "b5aea91a-52ea-4bcf-a0ba-a767a2f575d6", "date": "2023-04-27T10:35:36", "productId": 706, "browser": "Chrome", "version": "11.2", "device": "tablet", "source": "google", "ip_address": "190.186.132.126", "landing_page": "http://hernandez.com/categorylogin.html"}
{"impressionId": "a12b8461-9b14-4fa5-9b4d-f1a659473194", "date": "2023-05-03T10:17:10", "productId": 711, "browser": "Firefox", "version": "6.6", "device": "tablet", "source": "organic", "ip_address": "88.27.95.227", "landing_page": "http://www.ramos-spears.org/tags/tags/categoryprivacy.php"}
{"impressionId": "a12b8461-9b14-4fa5-9b4d-f1a659473194", "date": "2023-05-03T10:17:10", "productId": 711, "browser": "Firefox", "version": "6.6", "device": "tablet", "source": "organic", "ip_address": "88.27.95.227"}
{"impressionId": "abde85f7-704c-43af-a4ad-c604dd72adca", "date": "2023-01-30T14:06:58", "productId": 707, "browser": "Chrome", "version": "10.2", "device": "mobile", "source": "organic", "ip_address": "111.88.21

In [ ]:
isunsupportedBrowser = True
generateEvents(isunsupportedBrowser)